In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [82]:
#計測地点のosm_idを指定
start_osm = 306736550
mid_osm = [444834048,444834047]
end_osm = 306736592
start_mesh = 50326613421
end_mesh = 50326613241

In [83]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [84]:
dbname = "kddi_18_param_copy"
sttime = "2022-04-01 18:00:00"
dbname = "kddi_trkn_change2_f6"

In [85]:

datedir = "230206"
f = open(datedir+'/'+ dbname + '.csv', 'w')

for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_18_param_copy  "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        sql += "ORDER BY trakanmap.gridcode"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, param) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                #cur.execute(sql)
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, param) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + '\n')
        #cur.execute(sql)
        #connection.commit()
                
        cur.close()
        #print(lstItems)
        dte = dte + datetime.timedelta(minutes=15)

33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値78.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値78.0距離0.0 km
50326613243
存在メッシュ：50326613243値78.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値78.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値78.0距離0.0 km
50326613241
存在メッシュ：50326613241値78.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値78.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値78.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値78.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値80.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値80.0距離0.0 km
50326613243
存在メッシュ：50326613243値80.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値80.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値80.0距離0.0 km
50326613241
存在メッシュ：50326613241値80.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値80.0距離0.11554123617614287 

50326613243
存在メッシュ：50326613243値19.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値19.0距離0.0 km
50326613243
存在メッシュ：50326613243値19.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値19.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値19.0距離0.0 km
50326613241
存在メッシュ：50326613241値19.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値19.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値19.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値19.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値11.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値11.0距離0.0 km
50326613243
存在メッシュ：50326613243値11.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値11.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値11.0距離0.0 km
50326613241
存在メッシュ：50326613241値11.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値11.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値11.0距離0.23108245279765

50326613243
存在メッシュ：50326613243値31.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値31.0距離0.0 km
50326613243
存在メッシュ：50326613243値31.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値31.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値31.0距離0.0 km
50326613241
存在メッシュ：50326613241値31.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値31.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値31.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値31.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値37.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値37.0距離0.0 km
50326613243
存在メッシュ：50326613243値37.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値37.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値37.0距離0.0 km
50326613241
存在メッシュ：50326613241値37.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値37.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値37.0距離0.23108245279765

50326613243
存在メッシュ：50326613243値24.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値24.0距離0.0 km
50326613243
存在メッシュ：50326613243値24.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値24.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値24.0距離0.0 km
50326613241
存在メッシュ：50326613241値24.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値24.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値24.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値24.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値20.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値20.0距離0.0 km
50326613243
存在メッシュ：50326613243値20.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値20.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値20.0距離0.0 km
50326613241
存在メッシュ：50326613241値20.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値20.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値20.0距離0.23108245279765

50326613243
存在メッシュ：50326613243値26.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値26.0距離0.0 km
50326613243
存在メッシュ：50326613243値26.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値26.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値26.0距離0.0 km
50326613241
存在メッシュ：50326613241値26.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値26.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値26.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値26.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値26.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値26.0距離0.0 km
50326613243
存在メッシュ：50326613243値26.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値26.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値26.0距離0.0 km
50326613241
存在メッシュ：50326613241値26.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値26.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値26.0距離0.23108245279765

...

50326613243
50326613243 is kddi none!
50326613421
存在メッシュ：50326613421値41.0距離0.0 km
50326613243
50326613243 is kddi none!
50326613241
存在メッシュ：50326613241値41.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値41.0距離0.0 km
50326613241
存在メッシュ：50326613241値41.0距離0.2310824527976543 km
50326613243
50326613243 is kddi none!
50326613241
存在メッシュ：50326613241値41.0距離0.2310824527976543 km
50326613243
50326613243 is kddi none!
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値40.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値40.0距離0.0 km
50326613243
存在メッシュ：50326613243値40.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値40.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値40.0距離0.0 km
50326613241
存在メッシュ：50326613241値40.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値40.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値40.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値40.0距離0.11554123617614287 km
33.8442708335:33.84635

50326613243
存在メッシュ：50326613243値15.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値15.0距離0.0 km
50326613243
存在メッシュ：50326613243値15.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値15.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値15.0距離0.0 km
50326613241
存在メッシュ：50326613241値15.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値15.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値15.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値15.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値10.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値10.0距離0.0 km
50326613243
存在メッシュ：50326613243値10.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値10.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値10.0距離0.0 km
50326613241
存在メッシュ：50326613241値10.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値10.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値10.0距離0.23108245279765

50326613243
存在メッシュ：50326613243値19.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値19.0距離0.0 km
50326613243
存在メッシュ：50326613243値19.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値19.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値19.0距離0.0 km
50326613241
存在メッシュ：50326613241値19.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値19.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値19.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値19.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値25.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値25.0距離0.0 km
50326613243
存在メッシュ：50326613243値25.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値25.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値25.0距離0.0 km
50326613241
存在メッシュ：50326613241値25.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値25.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値25.0距離0.23108245279765

50326613243
存在メッシュ：50326613243値44.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値44.0距離0.0 km
50326613243
存在メッシュ：50326613243値44.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値44.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値44.0距離0.0 km
50326613241
存在メッシュ：50326613241値44.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値44.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値44.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値44.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値39.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値39.0距離0.0 km
50326613243
存在メッシュ：50326613243値39.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値39.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値39.0距離0.0 km
50326613241
存在メッシュ：50326613241値39.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値39.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値39.0距離0.23108245279765

50326613243
存在メッシュ：50326613243値12.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値12.0距離0.0 km
50326613243
存在メッシュ：50326613243値12.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値12.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値12.0距離0.0 km
50326613241
存在メッシュ：50326613241値12.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値12.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値12.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値12.0距離0.11554123617614287 km
33.8442708335:33.8463541665:132.797656250:132.797656250
50326613243
存在メッシュ：50326613243値12.0距離0.11554123617614287 km
50326613421
存在メッシュ：50326613421値12.0距離0.0 km
50326613243
存在メッシュ：50326613243値12.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値12.0距離0.2310824527976543 km
50326613421
存在メッシュ：50326613421値12.0距離0.0 km
50326613241
存在メッシュ：50326613241値12.0距離0.2310824527976543 km
50326613243
存在メッシュ：50326613243値12.0距離0.11554123617614287 km
50326613241
存在メッシュ：50326613241値12.0距離0.23108245279765

In [86]:
connection.close()

In [56]:
#CreateGeojsonFile

In [87]:
f.close()